In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder

In [5]:
# Load the dataset
df = pd.read_csv('./food_recipes_2.csv')

X = df[['ingredients1', 'ingredients2', 'ingredients3', 'ingredients4', 'ingredients5',
        'ingredients6', 'ingredients7', 'ingredients8', 'ingredients9', 'ingredients10']]
y = df['recipe_title']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Fill nulls with empty strings
X_train = X_train.fillna('')
X_test = X_test.fillna('')
y_train = y_train.fillna('')  
y_test = y_test.fillna('')

In [6]:
# Text vectorization using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [7]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=64, input_length=X_train_vec.shape[1]))
model.add(LSTM(100, return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dense(50, activation='relu'))
model.add(Dense(len(df['recipe_title'].unique()), activation='softmax'))

In [8]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Convert labels to numerical format
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

In [10]:
# Train the model
model.fit(X_train_vec.toarray(), y_train_encoded, epochs=5, batch_size=64, validation_split=0.1)

Epoch 1/5


1/1 [==============================] - 8s 8s/step - loss: 8.8159 - accuracy: 0.0000e+00 - val_loss: 8.8172 - val_accuracy: 0.0000e+00
Epoch 2/5
1/1 [==============================] - 0s 79ms/step - loss: 8.8128 - accuracy: 0.1111 - val_loss: 8.8177 - val_accuracy: 0.0000e+00
Epoch 3/5
1/1 [==============================] - 0s 58ms/step - loss: 8.8095 - accuracy: 0.1111 - val_loss: 8.8185 - val_accuracy: 0.0000e+00
Epoch 4/5
1/1 [==============================] - 0s 50ms/step - loss: 8.8057 - accuracy: 0.1111 - val_loss: 8.8195 - val_accuracy: 0.0000e+00
Epoch 5/5
1/1 [==============================] - 0s 66ms/step - loss: 8.8013 - accuracy: 0.1111 - val_loss: 8.8206 - val_accuracy: 0.0000e+00


In [11]:
# Evaluate the model on the test set
y_test_encoded = label_encoder.fit_transform(y_test)

In [14]:
loss, accuracy = model.evaluate(X_test_vec.toarray(), y_test_encoded)

print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

ValueError: Data cardinality is ambiguous:
  x sizes: 10
  y sizes: 1349
Make sure all arrays contain the same number of samples.